In [ ]:
import scanpy as sc
import besca as bc
import os
import bbknn
import numpy as np
import scvelo as scv
import pandas as pd

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as ss
from scipy import stats
import itertools


In [ ]:
color_dict = {'R': 'coral', 'TF': 'firebrick', 'R_nadj': 'lightskyblue','R_adj': 'royalblue'}


In [ ]:
sc.logging.print_versions()

# Reading the h5ad

In [ ]:
WD='/Fullanalysis/'
analysis_name='sw_besca2_immune'
dcdata=sc.read(WD+'analyzed/'+analysis_name+'/'+analysis_name+'.annotated.filtered.h5ad')
figdir='/Fullanalysis/figures/'

In [ ]:
outpath=WD+'/analyzed/sw_besca2_immune/velocity/'

In [ ]:
velosubset='CD4TnoTregnoProlif'

In [ ]:
sc.set_figure_params()

In [ ]:
#sc.pl.umap(dcdata, color=['leiden', 'leiden_TNK', 'Sample type'],legend_loc='on data', legend_fontsize=0.6)

In [ ]:
sc.set_figure_params()
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,10)

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_rc_cells'], size=8)

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_pub'], size=8)

In [ ]:
sc.set_figure_params()
sc.pl.umap(dcdata,color=['CD8A','CD8B','CD4','CD3D'], color_map='viridis')

In [ ]:
cd4=dcdata[(dcdata.obs['celltype2_rc_cells'].isin(['CD4-positive, alpha-beta T cell']))].copy()
cd4=cd4[(cd4.obs['celltype2_pub'].isin(['CD4-positive T cell']))].copy()
cd8=dcdata[(dcdata.obs['celltype2_rc_cells'].isin(['CD8-positive, alpha-beta T cell']))].copy()
cd8=cd8[(cd8.obs['celltype2_pub'].isin(['CD8-positive T cell']))].copy()
myelo=dcdata[(dcdata.obs['celltype1_rc_cells'].isin(['myeloid leukocyte']))].copy()
myelo=myelo[(myelo.obs['celltype1'].isin(['myeloid leukocyte']))].copy()

In [ ]:
### Breakdown of cell types per experiment (sample)
bc.pl.celllabel_quant_stackedbar(
    cd4, count_variable="celltype3_pub", subset_variable="Lesion"
)

In [ ]:
### Breakdown of cell types per experiment (sample)
bc.pl.celllabel_quant_stackedbar(
    cd8, count_variable="celltype3_pub", subset_variable="Lesion"
)

In [ ]:
set(dcdata.obs['celltype2_merged'])

In [ ]:
set(dcdata.obs['celltype2_pub'])

In [ ]:
dcdata=dcdata[(dcdata.obs['celltype2_rc_cells'].isin(['CD4-positive, alpha-beta T cell']))].copy()
dcdata=dcdata[(dcdata.obs['celltype2_pub'].isin(['CD4-positive T cell']))].copy()


In [ ]:
dcdata.obs['leidenf']=dcdata.obs['leiden'].copy()

In [ ]:
sc.pl.umap(dcdata, color=['leiden', 'celltype3_merged','celltype3', 'Sample type'])

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_merged'],legend_fontsize=8) #14, 34, 3, 31


In [ ]:
sc.pl.umap(dcdata, color=['celltype2_pub'],legend_fontsize=8) #14, 34, 3, 31

In [ ]:
sc.pl.umap(dcdata, color=['celltype3_pub'],legend_fontsize=8) #14, 34, 3, 31


In [ ]:
sc.pl.umap(dcdata, color=['CD8A','CD4','GZMB','CCR7','XCL1','KLRB1','CXCR6','CCR6','SLC4A10','LTK'])

In [ ]:
sc.pl.umap(dcdata, color=['celltype2_rc_cells'],legend_fontsize=8) #14, 34, 3, 31


In [ ]:
path1='/looms/'
myc=['P11_digest_CD45pos.loom', 'P11_PBMC.loom', 'P12_digest_CD45pos.loom', 'P12_PBMC.loom', 
     'P13_digest_CD45pos.loom', 'P13_PBMC.loom', 'P29_digest_CD45pos.loom', 
     'P29_PBMC.loom', 'P33_digest_CD45pos.loom', 'P33_PBMC.loom', 'P34_digest_CD45pos.loom',
     'P34_PBMC.loom' ,'P38_digest_CD45pos.loom', 
     'P38_PBMC.loom', 'P40_digest_CD45pos.loom', 'P40_PBMC.loom' ,'P43_digest_2_CD45pos.loom', 
     'P43_digest_CD45pos.loom', 'P43_PBMC.loom', 'P63_digest_CD45pos.loom', 'P63_PBMC.loom', 
     'P64_digest_2_CD45pos.loom', 'P64_digest_CD45pos.loom', 'P67_digest_2_CD45pos.loom', 
     'P67_digest_CD45pos.loom' ,'P67_PBMC.loom' ,'P68_digest_CD45pos.loom' ,'P68_PBMC.loom', 
     'P69_digest_CD45pos.loom', 'P72_digest_CD45pos.loom', 'P72_PBMC.loom', 'P77_digest_CD45pos.loom', 
     'P79_digest_2_CD45pos.loom', 'P79_digest_CD45pos.loom', 'P79_PBMC.loom', '
     'P82_digest_CD45pos_and_CD45neg.loom', 'P82_PBMC.loom P83_digest_CD45pos.loom', 
     'P83_PBMC.loom', 'P86_digest_CD45pos.loom', 'P86_PBMC.loom', 'P87_digest_CD45pos.loom', 
     'P87_PBMC.loom', 'P91_digest_CD45pos.loom', 'P91_PBMC.loom']
m1=myc.copy()
myck=myc.copy() 

In [ ]:
loom=scv.read(WD+'/looms/loom_afterconcat.h5ad')

In [ ]:
#sc.pp.neighbors(dcdata)
bbknn.bbknn(dcdata, batch_key='PatientID', use_rep='X_pca')

In [ ]:
sc.tl.diffmap(dcdata)


sc.pp.neighbors(dcdata, n_neighbors=15, use_rep='X_diffmap')
sc.tl.leiden(dcdata, resolution=0.25)


sc.tl.paga(dcdata, groups='leiden')

sc.pl.umap(dcdata,color='leiden')


In [ ]:
sc.pl.paga(dcdata)

In [ ]:

sc.tl.umap(dcdata, init_pos='paga')

sc.pl.umap(dcdata,color=['leiden','leidenf'],legend_loc='on data')

In [ ]:
sc.pl.umap(dcdata,color=['celltype3'])

In [ ]:
#scv.utils.clean_obs_names(dcdata)
#scv.utils.clean_obs_names(loom)

In [ ]:
adata=scv.utils.merge(dcdata, loom).copy()

In [ ]:
#adata.write(WD+"/looms/CD4Tcells-"+velosubset+"loom.h5ad", compression="gzip")


In [ ]:
adata=sc.read(WD+"/looms/CD4Tcells-"+velosubset+"loom.h5ad")

In [ ]:
sc.set_figure_params(6)

In [ ]:
sc.pl.umap(adata,color=['XCL1','CCR7','SELL','MKI67', 'STMN1','ENTPD1','PRF1','GZMB','TIGIT','IL2RA','FOXP3',
                        'LAG3','CD38','TOX','CD4','CD8A','CD8B', 'CD3D'], 
           color_map='viridis',save='-subsettrajectorymap-'+velosubset+'.Markers1.svg')

In [ ]:
sc.pl.umap(adata, color=['celltype3_pub'],legend_fontsize=8, 
          save='-subsettrajectorymap-'+velosubset+'.celltype3_pub.svg') #14, 34, 3, 31


In [ ]:
sc.pl.umap(adata, color=['celltype3_pub'],legend_fontsize=8, 
          save='-subsettrajectorymap-'+velosubset+'.celltype3_pub.pdf') #14, 34, 3, 31


In [ ]:
figdir

In [ ]:
sc.pl.umap(adata, color=['PatientID'],legend_fontsize=8,
          save='-subsettrajectorymap-'+velosubset+'.PatientID.svg') #14, 34, 3, 31


In [ ]:
sc.pl.umap(adata, color=['Sample type'],legend_fontsize=8,
          save='-subsettrajectorymap-'+velosubset+'.SampleType.svg') #14, 34, 3, 31


In [ ]:
sc.pl.umap(adata,color=['leiden'],legend_loc='on data',
          save='-subsettrajectorymap-'+velosubset+'.leiden.svg')

In [ ]:
sc.pl.umap(adata,color=['RCat'],ncols=2,save='-subsettrajectorymap-'+velosubset+'.RCat.svg')

In [ ]:
sc.pl.umap(adata,color=['Lesion'],ncols=2,save='-subsettrajectorymap-'+velosubset+'.Lesion.svg')

In [ ]:
#velosubset

In [ ]:
#adata=scv.read(WD+"analyzed/sw_besca2_immune/velocity/Velo-DCs-All-PBMCandTIL-dyn.h5ad")

In [ ]:
#adata=scv.read(WD+"CD8Tcells-loom.h5ad")

In [ ]:
sc.set_figure_params(6)
#sc.pl.umap(adata,color=['PatientID'],ncols=2)

In [ ]:
sc.pl.umap(adata,color=['leiden'],ncols=2, legend_loc='on data')

In [ ]:
scv.pl.proportions(adata)

In [ ]:
scv.pl.proportions(adata, groupby='leiden')

In [ ]:
subdata=adata.copy() #[adata.obs.sample(frac=0.5).index].copy()

In [ ]:
#subdata=subdata[subdata.obs.sample(frac=0.5).index].copy()#[adata.obs.sample(frac=0.5).index].copy()

In [ ]:
sc.pl.umap(subdata,color=['leiden'],ncols=2, legend_loc='on data')

In [ ]:
#subdata=adata.copy()

In [ ]:
sc.pl.umap(adata,color=['Lesion'],ncols=2)

In [ ]:
adata.obs['PatientID']=adata.obs['PatientID'].astype('str')

In [ ]:
bc.pl.celllabel_quant_boxplot(
    adata,
    count_variable="leiden",
    subset_variable="PatientID",
    condition_identifier="RCat",
    plot_percentage=True,
)

In [ ]:
myperc=bc.tl.count_occurrence_subset_conditions(
    adata,
    'PatientID',
    'Lesion',
    count_variable="leiden",
    return_percentage=True,
)

In [ ]:
myperc=bc.tl.count_occurrence_subset_conditions(
    adata,
    'Lesion',
    'Sample type',
    count_variable="leiden",
    return_percentage=True,
)

In [ ]:
myperc

In [ ]:
figdir+'Percentage-Lesion-'+velosubset+'.all.PatientID.svg'

In [ ]:
### Breakdown of cell types per experiment (sample)
bc.pl.celllabel_quant_stackedbar(
    adata, count_variable="leiden", subset_variable="Lesion"
)
plt.savefig(figdir+'Percentage-Lesion-'+velosubset+'.all.PatientID.svg', format='svg', bbox_inches="tight", dpi=300)


In [ ]:
### Breakdown of cell types per experiment (sample)
bc.pl.celllabel_quant_stackedbar(
    adata, count_variable="leiden", subset_variable="RCat"
)
plt.savefig(figdir+'Percentage-RCat-'+velosubset+'.all.PatientID.svg', format='svg', bbox_inches="tight", dpi=300)


In [ ]:
### Breakdown of cell types per experiment (sample)
bc.pl.celllabel_quant_stackedbar(
    adata, count_variable="leiden", subset_variable="PatientID"
)
plt.savefig(figdir+'Percentage-Patient-'+velosubset+'.all.PatientID.svg', format='svg', bbox_inches="tight", dpi=300)


In [ ]:
sc.pl.umap(subdata,color=['CD8A','CD8B','CD4'],ncols=3, color_map='viridis')

In [ ]:
#catorder=['8','12','10','6','4','5','3','1']
#catorder=corder=['2','5','10','7','13','8']

In [ ]:
#adata=adata[adata.obs['leiden'].isin(catorder)].copy()

In [ ]:
scv.pp.filter_and_normalize(subdata,min_shared_counts=20)
scv.pp.neighbors(subdata)
scv.pp.moments(subdata, n_pcs=30, n_neighbors=30)



In [ ]:
scv.tl.recover_dynamics(subdata, n_jobs=20)

In [ ]:
scv.tl.velocity(subdata,mode='dynamical')
scv.tl.velocity_graph(subdata)

In [ ]:
#scv.utils.randomized_velocity(subdata)

In [ ]:
scv.tl.velocity_confidence(subdata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(subdata, c=keys, cmap='coolwarm', perc=[5, 95], save='Velo-len-conf-allCD4T.svg')

In [ ]:
#sc.set_figure_params(10)
#import matplotlib.pyplot as plt
#plt.rcParams["figure.figsize"] = (10,8)
scv.pl.velocity_embedding_stream(subdata, basis='umap',color=['Sample type'], 
                                 save='-velo-'+velosubset+'-dyn-Sample_type.svg')

In [ ]:
scv.pl.velocity_embedding_stream(subdata, basis='umap',color=['leiden'],
                                 save='-velo-'+velosubset+'-dyn-leiden.svg')


In [ ]:
scv.tl.recover_latent_time(subdata)

scv.pl.scatter(subdata, color='latent_time', color_map='gnuplot', size=80)

sc.pl.umap(subdata, color=['root_cells','end_points'])



In [ ]:
scv.pl.scatter(subdata, color='velocity_pseudotime', color_map='gnuplot', size=80,
              save='-velo-'+velosubset+'-pseudotime.svg')


In [ ]:
top_genes = subdata.var['fit_likelihood'].sort_values(ascending=False).index
scv.pl.scatter(subdata, basis=top_genes[:15], color='leiden',ncols=5, frameon=False)



In [ ]:
subdata.obs.loc[((subdata.obs['root_cells']>0.95) & (subdata.obs['Sample type']  == 'PBMC')),'leiden'].value_counts()


In [ ]:
subdata.obs.loc[((subdata.obs['end_points']>0.9) & (subdata.obs['Sample type']  == 'TIL')),'leiden'].value_counts()


In [ ]:
subdata.uns['iroot'] = np.flatnonzero(((subdata.obs['leiden']  == '0')&(subdata.obs['root_cells']>0.95)&(subdata.obs['Sample type']  == 'PBMC')))[0]
subdata.uns['iend'] = np.flatnonzero(((subdata.obs['leiden']  == '5')&(subdata.obs['end_points']>0.9)&(subdata.obs['Sample type']  == 'TIL')))[0]

scv.tl.recover_latent_time(subdata, root_key='iroot', end_key='iend')



In [ ]:
scv.pl.scatter(subdata, color=['latent_time','velocity_pseudotime'], color_map='gnuplot', size=80, save='-velo-'+velosubset+'-pseudotime-latent.svg')

In [ ]:
scv.tl.paga(subdata, groups='leiden', root_key='iroot', end_key='iend', 
            use_time_prior='velocity_pseudotime', threshold_root_end_prior=0.8)


sc.set_figure_params(10)
scv.pl.paga(subdata,cluster_key="leiden", basis='umap',node_size_scale=1, edge_width_scale=1, )

In [ ]:
#subcat='TILonly'
subdata.write(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-dyn.calculatedVelo.h5ad')

#### Read DE genes ####

In [ ]:
cdde=['CD4T-all-TILonly',
     "CD4T-AllnoTregMAIT-TILonly","CD4T-effector-TILonly","CD4T-naiveCM-TILonly"]
ffigdir='/Fullanalysis/analyzed/sw_besca2_immune/figures/publication/'

In [ ]:
alldesR=pd.DataFrame(None)
alldesNR=pd.DataFrame(None)
for mycd in cdde:

    cdcNRgenes=pd.read_csv(ffigdir+'DEanalysis_wilcoxon.'+mycd+'.NRhigh.tsv',sep='\t')
    cdcRgenes=pd.read_csv(ffigdir+'DEanalysis_wilcoxon.'+mycd+'.Rhigh.tsv',sep='\t')

    cdcNRgenes['Comparison']=mycd
    cdcRgenes['Comparison']=mycd
    alldesR=pd.concat([alldesR, cdcRgenes])
    alldesNR=pd.concat([alldesNR, cdcNRgenes])

In [ ]:
alldesR

In [ ]:
alldesNR

#### Run again focusing on main transition clusters ####

In [ ]:
velosubset='subCD4TnoTregnoProlif_v2'

In [ ]:
#sc.pl.umap(subdata,color=['leiden'],ncols=2, legend_loc='on data')

In [ ]:
subdata=adata.copy() #[adata.obs.sample(frac=0.5).index].copy()

###subdata=subdata[subdata.obs['leiden']!='5'].copy()
subdata=subdata[subdata.obs['leiden']!='14'].copy()
#subdata=subdata[subdata.obs['leiden']!='9'].copy()
subdata=subdata[subdata.obs['leiden']!='7'].copy()
subdata=subdata[subdata.obs['leiden']!='3'].copy()
subdata=subdata[subdata.obs['leiden']!='4'].copy()
subdata=subdata[subdata.obs['leiden']!='12'].copy()
#subdata=subdata[subdata.obs['leiden']!='10'].copy()

#subdata=subdata[subdata.obs.sample(frac=0.5).index].copy()#[adata.obs.sample(frac=0.5).index].copy()



In [ ]:
sc.pl.umap(subdata,color=['leiden'],ncols=2, legend_loc='on data',
           save='-subsettrajectorymap-'+velosubset+'.Leiden.svg')

In [ ]:
sc.pl.umap(subdata,color=['celltype3_pub'],ncols=2,save='-subsettrajectorymap-'+velosubset+'.celltype3_pub.svg')

In [ ]:
sc.pl.umap(subdata,color=['CD8A','CD4','FOXP3','MKI67'])

In [ ]:
### get subsets of the data per response ###
#rdata=subdata[subdata.obs['RCat'].isin(['R'])].copy()
#tfdata=subdata[subdata.obs['RCat'].isin(['TF'])].copy()
#rbdata=subdata[subdata.obs['RCat'].isin(['R', 'TF'])].copy()

#nrdata=subdata[subdata.obs['RCat'].isin(['NR_adj'])].copy()
#nrnadjdata=subdata[subdata.obs['RCat'].isin(['NR_nadj'])].copy()
#nrbdata=subdata[subdata.obs['RCat'].isin(['NR_adj','NR_nadj'])].copy()

In [ ]:
scv.pp.filter_and_normalize(subdata,min_shared_counts=20, n_top_genes=800)
scv.pp.neighbors(subdata)
scv.pp.moments(subdata, n_pcs=30, n_neighbors=30)

scv.tl.recover_dynamics(subdata, n_jobs=20)

scv.tl.velocity(subdata,mode='dynamical')
scv.tl.velocity_graph(subdata)


In [ ]:
#scv.tl.velocity_graph(subdata, mode_neighbors='connectivities') 

In [ ]:

#scv.utils.randomized_velocity(subdata)

scv.tl.velocity_confidence(subdata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(subdata, c=keys, cmap='coolwarm', perc=[5, 95])



In [ ]:
scv.pl.velocity_embedding_stream(subdata, basis='umap',color=['Sample type'], 
                                 save='-velo-'+velosubset+'-dyn-Sample_type.png')

In [ ]:
#sc.set_figure_params(10)
#import matplotlib.pyplot as plt
#plt.rcParams["figure.figsize"] = (10,8)
scv.pl.velocity_embedding_stream(subdata, basis='umap',color=['Sample type'], 
                                 save='-velo-'+velosubset+'-dyn-Sample_type.svg')

In [ ]:
scv.pl.velocity_embedding_stream(subdata, basis='umap',color=['leiden'], 
                                 save='-velo-'+velosubset+'-dyn-leiden.png')

In [ ]:
scv.pl.velocity_embedding_stream(subdata, basis='umap',color=['leiden'], 
                                 save='-velo-'+velosubset+'-dyn-leiden.svg')

In [ ]:
### No initialisation latent time 

scv.tl.recover_latent_time(subdata)
scv.pl.scatter(subdata, color='latent_time', color_map='gnuplot', size=80)

In [ ]:
sc.pl.umap(subdata, color=['root_cells','end_points'])

In [ ]:
scv.pl.scatter(subdata, color='velocity_pseudotime', color_map='gnuplot', size=80)

In [ ]:
top_genes = subdata.var['fit_likelihood'].sort_values(ascending=False).index
scv.pl.scatter(subdata, basis=top_genes[:15], color='leiden',ncols=5, frameon=False)

In [ ]:
subdata.obs.loc[((subdata.obs['root_cells']>0.38) & (subdata.obs['Sample type']  == 'PBMC')),'leiden'].value_counts()

In [ ]:
subdata.obs.loc[((subdata.obs['end_points']>0.9) & (subdata.obs['Sample type']  == 'TIL')),'leiden'].value_counts()

In [ ]:
subdata.uns['iroot'] = np.flatnonzero(((subdata.obs['leiden']  == '0')&(subdata.obs['root_cells']>0.38)&(subdata.obs['Sample type']  == 'PBMC')))[0]
subdata.uns['iend'] = np.flatnonzero(((subdata.obs['leiden']  == '5')&(subdata.obs['end_points']>0.9)&(subdata.obs['Sample type']  == 'TIL')))[0]

scv.tl.recover_latent_time(subdata, root_key='iroot', end_key='iend')

In [ ]:
scv.pl.scatter(subdata, color=['latent_time','velocity_pseudotime'], color_map='gnuplot', size=80,
               save='-velo-'+velosubset+'-latent_time.svg')

#### Reanalyse per response subset ####

In [ ]:
rbdata_new=subdata[subdata.obs.RCat.isin(['TF','R'])].copy()

In [ ]:
nrbdata_new=subdata[subdata.obs.RCat.isin(['NR_adj','NR_nadj'])].copy()

In [ ]:
scv.pp.neighbors(rbdata_new)


In [ ]:
scv.pp.moments(rbdata_new, n_pcs=30, n_neighbors=30)

scv.tl.recover_dynamics(rbdata_new, n_jobs=20)

scv.tl.velocity(rbdata_new,mode='dynamical')

In [ ]:
scv.tl.velocity_graph(rbdata_new)

In [ ]:
scv.tl.paga(rbdata_new, groups='leiden', root_key='iroot', end_key='iend', 
            use_time_prior='velocity_pseudotime', threshold_root_end_prior=0.8)


In [ ]:
sc.set_figure_params(10)
scv.pl.paga(rbdata_new,cluster_key="leiden", basis='umap',node_size_scale=1, edge_width_scale=1,
           save='-paga-'+velosubset+'-rbdata-dyn-leiden.svg')

In [ ]:
scv.pl.scatter(rbdata_new, color=['latent_time','velocity_pseudotime'], color_map='gnuplot', size=80,
              save='-latent_time-'+velosubset+'-rbdata-dyn-leiden.svg')

In [ ]:
scv.pl.velocity_embedding_stream(rbdata_new, basis='umap',color=['Sample type'], 
                                save='-velo-'+velosubset+'-rbdata-dyn-sample_type.svg')

In [ ]:
rbdata_new.obs['clusters']=rbdata_new.obs['leiden'].copy()

In [ ]:
top_genes = rbdata_new.var['fit_likelihood'].sort_values(ascending=False).index[:50]
scv.pl.heatmap(rbdata_new, var_names=top_genes, sortby='latent_time', col_color='clusters',
               n_convolve=100,font_scale=0.6)

In [ ]:
rbgenes=rbdata_new.var.loc[rbdata_new.var['velocity_genes']==True,:]['fit_likelihood'].sort_values(ascending=False)

In [ ]:
nrbdata_new.obs['clusters']=nrbdata_new.obs['leiden'].copy()

In [ ]:
scv.pp.neighbors(nrbdata_new)

scv.pp.moments(nrbdata_new, n_pcs=30, n_neighbors=30)

scv.tl.recover_dynamics(nrbdata_new, n_jobs=20)

scv.tl.velocity(nrbdata_new,mode='dynamical')


In [ ]:

scv.tl.velocity_graph(nrbdata_new)

scv.tl.paga(nrbdata_new, groups='leiden', root_key='iroot', end_key='iend', 
            use_time_prior='velocity_pseudotime', threshold_root_end_prior=0.8)


sc.set_figure_params(10)
scv.pl.paga(nrbdata_new,cluster_key="leiden", basis='umap',node_size_scale=1, edge_width_scale=1, 
           save='-paga-'+velosubset+'-nrbdata-dyn-leiden.svg')

In [ ]:
scv.pl.velocity_embedding_stream(nrbdata_new, basis='umap',color=['Sample type'],
                                save='-velo-'+velosubset+'-nrbdata-dyn-sample_type.svg')

In [ ]:
nrbgenes=nrbdata_new.var.loc[nrbdata_new.var['velocity_genes']==True,:]['fit_likelihood'].sort_values(ascending=False)

In [ ]:
velogenes=pd.concat([rbgenes, nrbgenes], axis=1)
velogenes.columns=['R','NR']

In [ ]:
velogenes.sort_values(by='NR', ascending=False)

In [ ]:
rbgenes

In [ ]:
sc.pl.umap(rbdata_new,color='leiden')

In [ ]:
velogenes.to_csv(WD+'analyzed/'+analysis_name+'/velocity/Velo-genes-per-response-CD4T.tsv', sep='\t')

In [ ]:
top_genes=set(alldesR['Unnamed: 0']).intersection(set(rbgenes[rbgenes>0.1].index))

In [ ]:
top_genes

In [ ]:
sc.pl.umap(rbdata_new,color=top_genes)

In [ ]:
sc.pl.umap(rbdata_new,color=['ITGB1','BIRC5','CTLA4','ICA1'], ncols=2,
           save='-rbdata_new-'+velosubset+'-velogenes.svg')

In [ ]:
scv.pl.heatmap(rbdata_new, var_names=top_genes, sortby='latent_time', col_color='leiden',
               n_convolve=100,font_scale=0.6,  save='-rbdata_new-'+velosubset+'.latentimeVeloGenes.Leiden.svg')

In [ ]:
top_genes=set(alldesNR['Unnamed: 0']).intersection(set(nrbgenes[nrbgenes>0.1].index))

In [ ]:
top_genes

In [ ]:
sc.pl.umap(nrbdata_new,color=top_genes)

In [ ]:
sc.pl.umap(nrbdata_new,color=['NR4A2','NFKBIA','FOSB','ZNF331'], ncols=2,
           save='-nrbdata_new-'+velosubset+'-velogenes.svg')

In [ ]:
scv.pl.heatmap(nrbdata_new, var_names=top_genes, sortby='latent_time', col_color='celltype3_pub',
               n_convolve=100,font_scale=0.6,save='-nrbdata_new-'+velosubset+'.latentimeVeloGenes.Leiden.svg')

#### For the top R patient check velocity 

In [ ]:
bc.tl.count_occurrence(rbdata_new, count_variable="PatientID", add_percentage=False)

In [ ]:
bc.tl.count_occurrence(nrbdata_new, count_variable="PatientID", add_percentage=False)

In [ ]:
p34=subdata[subdata.obs.PatientID=='34'].copy()
p34=p34[p34.obs['Sample type']=='TIL']

In [ ]:
scv.pp.neighbors(p34)


scv.pp.moments(p34, n_pcs=30, n_neighbors=30)

scv.tl.recover_dynamics(p34, n_jobs=20)

scv.tl.velocity(p34,mode='dynamical')

scv.tl.velocity_graph(p34)

scv.tl.paga(p34, groups='leiden', root_key='iroot', end_key='iend', 
            use_time_prior='velocity_pseudotime', threshold_root_end_prior=0.8)


sc.set_figure_params(10)
scv.pl.paga(p34,cluster_key="leiden", basis='umap',node_size_scale=1, edge_width_scale=1,
           save='-paga-'+velosubset+'-p34-dyn-leiden.svg')

In [ ]:
scv.pl.velocity_embedding_stream(p34, basis='umap',color=['Sample type'],
                                save='-velo-'+velosubset+'-p34-dyn-sample_type.svg')

In [ ]:
p67=subdata[subdata.obs.PatientID=='67'].copy()
p67=p67[p67.obs['Sample type']=='TIL']

scv.pp.neighbors(p67)


scv.pp.moments(p67, n_pcs=30, n_neighbors=30)

scv.tl.recover_dynamics(p67, n_jobs=20)

scv.tl.velocity(p67,mode='dynamical')

scv.tl.velocity_graph(p67)

scv.tl.paga(p67, groups='leiden', root_key='iroot', end_key='iend', 
            use_time_prior='velocity_pseudotime', threshold_root_end_prior=0.8)


sc.set_figure_params(10)
scv.pl.paga(p67,cluster_key="leiden", basis='umap',node_size_scale=1, edge_width_scale=1,
           save='-paga-'+velosubset+'-p67-dyn-leiden.svg')

scv.pl.velocity_embedding_stream(p67, basis='umap',color=['Sample type'],
                                save='-velo-'+velosubset+'-p67-dyn-sample_type.svg')

In [ ]:
p79=subdata[subdata.obs.PatientID=='79'].copy()
p79=p79[p79.obs['Sample type']=='TIL']

scv.pp.neighbors(p79)


scv.pp.moments(p79, n_pcs=30, n_neighbors=30)

scv.tl.recover_dynamics(p79, n_jobs=20)

scv.tl.velocity(p79,mode='dynamical')

scv.tl.velocity_graph(p79)

scv.tl.paga(p79, groups='leiden', root_key='iroot', end_key='iend', 
            use_time_prior='velocity_pseudotime', threshold_root_end_prior=0.8)


sc.set_figure_params(10)
scv.pl.paga(p79,cluster_key="leiden", basis='umap',node_size_scale=1, edge_width_scale=1,
           save='-paga-'+velosubset+'-p67-dyn-leiden.svg')

scv.pl.velocity_embedding_stream(p79, basis='umap',color=['Sample type'],
                                save='-velo-'+velosubset+'-p79-dyn-sample_type.svg')

In [ ]:
p64=subdata[subdata.obs.PatientID=='64'].copy()
p64=p64[p64.obs['Sample type']=='TIL']

In [ ]:


scv.pp.neighbors(p64)


scv.pp.moments(p64, n_pcs=30, n_neighbors=30)

scv.tl.recover_dynamics(p64, n_jobs=20)

scv.tl.velocity(p64,mode='dynamical')

scv.tl.velocity_graph(p64)

scv.tl.paga(p64, groups='leiden', root_key='iroot', end_key='iend', 
            use_time_prior='velocity_pseudotime', threshold_root_end_prior=0.8)


sc.set_figure_params(10)
scv.pl.paga(p64,cluster_key="leiden", basis='umap',node_size_scale=1, edge_width_scale=1,
           save='-paga-'+velosubset+'-p64-dyn-leiden.svg')

scv.pl.velocity_embedding_stream(p64, basis='umap',color=['Sample type'],
                                save='-velo-'+velosubset+'-p64-dyn-sample_type.svg')

In [ ]:
p77=subdata[subdata.obs.PatientID=='77'].copy()
p77

scv.pp.neighbors(p77)


scv.pp.moments(p77, n_pcs=30, n_neighbors=30)

scv.tl.recover_dynamics(p77, n_jobs=20)

scv.tl.velocity(p77,mode='dynamical')

scv.tl.velocity_graph(p77)

scv.tl.paga(p77, groups='leiden', root_key='iroot', end_key='iend', 
            use_time_prior='velocity_pseudotime', threshold_root_end_prior=0.8)


sc.set_figure_params(10)
scv.pl.paga(p77,cluster_key="leiden", basis='umap',node_size_scale=1, edge_width_scale=1,
           save='-paga-'+velosubset+'-p77-dyn-leiden.svg')

scv.pl.velocity_embedding_stream(p77, basis='umap',color=['Sample type'],
                                save='-velo-'+velosubset+'-p77-dyn-sample_type.svg')

In [ ]:
p33=subdata[subdata.obs.PatientID=='33'].copy()
p33=p33[p33.obs['Sample type']=='TIL']

scv.pp.neighbors(p33)


scv.pp.moments(p33, n_pcs=30, n_neighbors=30)

scv.tl.recover_dynamics(p33, n_jobs=20)

scv.tl.velocity(p33,mode='dynamical')

scv.tl.velocity_graph(p33)

scv.tl.paga(p33, groups='leiden', root_key='iroot', end_key='iend', 
            use_time_prior='velocity_pseudotime', threshold_root_end_prior=0.8)


sc.set_figure_params(10)
scv.pl.paga(p33,cluster_key="leiden", basis='umap',node_size_scale=1, edge_width_scale=1,
           save='-paga-'+velosubset+'-p33-dyn-leiden.svg')

scv.pl.velocity_embedding_stream(p33, basis='umap',color=['Sample type'],
                                save='-velo-'+velosubset+'-p33-dyn-sample_type.svg')

In [ ]:
p91=subdata[subdata.obs.PatientID=='91'].copy()
p91=p91[p91.obs['Sample type']=='TIL']

scv.pp.neighbors(p91)


scv.pp.moments(p91, n_pcs=30, n_neighbors=30)

scv.tl.recover_dynamics(p91, n_jobs=20)

scv.tl.velocity(p91,mode='dynamical')

scv.tl.velocity_graph(p91)

scv.tl.paga(p91, groups='leiden', root_key='iroot', end_key='iend', 
            use_time_prior='velocity_pseudotime', threshold_root_end_prior=0.8)


sc.set_figure_params(10)
scv.pl.paga(p91,cluster_key="leiden", basis='umap',node_size_scale=1, edge_width_scale=1,
           save='-paga-'+velosubset+'-p91-dyn-leiden.svg')

scv.pl.velocity_embedding_stream(p91, basis='umap',color=['Sample type'],
                                save='-velo-'+velosubset+'-p91-dyn-sample_type.svg')

In [ ]:
li={}
li['NaiTcell']=['CCR7','IL7R','TCF7']
li['MemTcell']=[ 'SELL', 'C1orf21', 'KLRB1', 'ARL4C']
li['TExh']=['TNFRSF9', 'CSF1', 'TIGIT']

sc.tl.score_genes(adata,li['NaiTcell'],score_name="LI_NAI",use_raw=True)
sc.tl.score_genes(adata,li['MemTcell'],score_name="LI_MEM",use_raw=True)
sc.tl.score_genes(adata,li['TExh'],score_name="LI_EXH",use_raw=True)

wu={}
wu['CD4Tcell']=['IL6ST','CRIP1']
wu['Teff']=['CX3CR1','GNLY', 'NKG7',  'GZMH', 'KLRD1', 'GZMB', 'PRF1', 
       'IFITM2', 'LITAF','ITGB2','GZMA','GPR56','KLRC2','GZMM','RAP1B'] ## KLRC2 is higher in Rs
wu['TEM']=['GZMK','CCL4',   'DUSP2', 'CD74','DNAJB1','FOS','CCL3','IFNG'] ## DNAJ1, DUSP2, GZMK is higher in NRs
wu['TRM']=['CCL4', 'XCL1',   'XCL2',   'ZNF683']
wu['IL17']=['NCR3','KLRB1','LYAR','IL7R']

sc.tl.score_genes(adata,wu['CD4Tcell'],score_name="WU_CD4",use_raw=True)
sc.tl.score_genes(adata,wu['Teff'],score_name="WU_TEFF",use_raw=True)
sc.tl.score_genes(adata,wu['TEM'],score_name="WU_TEM",use_raw=True)
sc.tl.score_genes(adata,wu['TRM'],score_name="WU_TRM",use_raw=True)
sc.tl.score_genes(adata,wu['IL17'],score_name="WU_IL17",use_raw=True)

In [ ]:
li={}
li['NaiTcell']=['CCR7','IL7R','TCF7']
li['MemTcell']=[ 'SELL', 'C1orf21', 'KLRB1', 'ARL4C']
li['TExh']=['TNFRSF9', 'CSF1', 'TIGIT']

sc.tl.score_genes(subdata,li['NaiTcell'],score_name="LI_NAI",use_raw=True)
sc.tl.score_genes(subdata,li['MemTcell'],score_name="LI_MEM",use_raw=True)
sc.tl.score_genes(subdata,li['TExh'],score_name="LI_EXH",use_raw=True)

wu={}
wu['CD4Tcell']=['IL6ST','CRIP1']
wu['Teff']=['CX3CR1','GNLY', 'NKG7',  'GZMH', 'KLRD1', 'GZMB', 'PRF1', 
       'IFITM2', 'LITAF','ITGB2','GZMA','GPR56','KLRC2','GZMM','RAP1B'] ## KLRC2 is higher in Rs
wu['TEM']=['GZMK','CCL4',   'DUSP2', 'CD74','DNAJB1','FOS','CCL3','IFNG'] ## DNAJ1, DUSP2, GZMK is higher in NRs
wu['TRM']=['CCL4', 'XCL1',   'XCL2',   'ZNF683']
wu['IL17']=['NCR3','KLRB1','LYAR','IL7R']

sc.tl.score_genes(subdata,wu['CD4Tcell'],score_name="WU_CD4",use_raw=True)
sc.tl.score_genes(subdata,wu['Teff'],score_name="WU_TEFF",use_raw=True)
sc.tl.score_genes(subdata,wu['TEM'],score_name="WU_TEM",use_raw=True)
sc.tl.score_genes(subdata,wu['TRM'],score_name="WU_TRM",use_raw=True)
sc.tl.score_genes(subdata,wu['IL17'],score_name="WU_IL17",use_raw=True)

In [ ]:
sc.pl.umap(adata, color=['LI_NAI','LI_MEM','LI_EXH','WU_CD4','WU_CD4','WU_TRM','WU_TEM','WU_TEFF','WU_IL17'])

In [ ]:
sc.pl.umap(adata, color=['LI_NAI','LI_MEM','LI_EXH'], save='-markers-'+velosubset+'-Li.svg')

In [ ]:
sc.pl.umap(subdata, color=['LI_NAI','LI_MEM','LI_EXH','WU_CD4','WU_CD4','WU_TRM','WU_TEM','WU_TEFF','WU_IL17'])

In [ ]:
sc.pl.umap(subdata, color=['BTLA','TNFRSF1B','HAVCR2','ANXA1'], save='-velo-'+velosubset+'-interactiongenes.svg')

In [ ]:
#subcat='TILonly'
subdata.write(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-dyn.calculatedVelo.final.h5ad')

In [ ]:
subdata=scv.read(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-dyn.calculatedVelo.final.h5ad')

In [ ]:
subdata.obs['celltype_new']=subdata.obs['leiden'].copy()

subdata.obs['celltype_new']=subdata.obs['celltype_new'].replace('1', 'central memory')
subdata.obs['celltype_new']=subdata.obs['celltype_new'].replace('0', 'central memory')
subdata.obs['celltype_new']=subdata.obs['celltype_new'].replace('11', 'central memory')
subdata.obs['celltype_new']=subdata.obs['celltype_new'].replace('2', 'effector memory')
subdata.obs['celltype_new']=subdata.obs['celltype_new'].replace('5', 'effector memory, cellular stress')
subdata.obs['celltype_new']=subdata.obs['celltype_new'].replace('13', 'activated, interferon')
subdata.obs['celltype_new']=subdata.obs['celltype_new'].replace('6', 'effector')
subdata.obs['celltype_new']=subdata.obs['celltype_new'].replace('8', 'follicular helper')
subdata.obs['celltype_new']=subdata.obs['celltype_new'].replace('9', 'proliferating')
subdata.obs['celltype_new']=subdata.obs['celltype_new'].replace('10', 'central memory/Treg')

In [ ]:
sc.pl.umap(subdata, color='leiden',legend_loc='on data')

In [ ]:
sc.pl.umap(subdata, color='celltype_new')

In [ ]:
sc.pl.umap(subdata, color='celltype3_pub')

In [ ]:
goi=['CDCA2','CDC20','DLGAP5','MKI67','BIRC5','AURKB','TYMS','STMN1',
     'CD40LG','TMIGD2','ACTN1','KLF2','PLAC8','TCF7','SELL','CCR7','IL7R',  'PTGER2',
      'PIK3R1','KLRG1','ZNF683','FCRL6','MATK','GNLY',
     'XCL1','XCL2','CD200','CXCR3','SLAMF6','IL2','IL18R1','CXCR5','ID3','S1PR1','KLF3','IFNG','CCL4','CCL3','CXCL13',
     'KLRF1','KLRB1','CRTAM','CD28','CAV1','IFITM1','CXCR6','KLRC2','ITGA1','CD101','CD69','FOS','GZMK','CD27',
     'HLA-DQA1','GZMB','GZMA','PRF1','TNFRSF9','PRDM1','TOX','TIGIT','LAG3','HAVCR2','PDCD1',
     'ENTPD1','CTLA4','ITGAE','CD38','DUSP4','DUSP1','ATF3','HSPA1A','HSPA1B']

In [ ]:
labeloi="leiden"

### Perform DE cells of each celltype3 vs. all other cells
DEgenes = bc.tl.dge.get_de(
    adata, labeloi, demethod="wilcoxon", topnr=5000, logfc=1, padj=0.05
)

In [ ]:
### Export top marker genes per celltype to .tsv file to facilitate checking the annotation
top5s=list()
for myc in DEgenes.keys():
    top5s=top5s+list(DEgenes[myc].sort_values("Log2FC", ascending=False).iloc[0:8,:]['Name'])
    DEgenes[myc].sort_values("Log2FC", ascending=False).iloc[0:50,:].to_csv(WD + "/figures/TopMarkergenes-" + labeloi + "_"+ myc + "_top50_wilcox.csv", sep="\t",index=False)

### Generate plots of the top marker genes 
sc.pl.dotplot(adata, var_names=top5s, groupby=labeloi, dot_max=0.8,vmax=3, dendrogram=True, save="TopMarkerplots-"+labeloi+".svg")



In [ ]:
sc.pl.matrixplot(adata, var_names=top5s, groupby=labeloi, standard_scale='var',  dendrogram=True, save="TopMarkerplots-"+labeloi+".svg")

In [ ]:
tmp=adata[adata.obs.leiden!='14']
#tmp=tmp[tmp.obs.leiden!='12']

In [ ]:
myorder=['4', '3', '1', '0', '11', '13','2','10', '7', '12',  '6', '8', '5']
myorders=[ '1', '0', '11', '13','2','10','9', '8', '6',  '5']

goi=['CDCA2','MKI67','STMN1','MAL',
     'TCF7','SELL','CCR7','IL7R','S1PR1','KLF3','IFIT1','RSAD2','ISG15','MX1','IFITM1',
       'KLRG1','ZNF683','FCRL6',
     'XCL1','IFNG','CCL4','CCL3',
     'KLRC2','ITGA1','CD69','FOS','GZMK','CCL5','GZMB','GZMA','PRF1','DUSP1','ATF3','HSPA1A','HSPA1B','PRDM1','LAG3','HAVCR2',
     'ENTPD1','CTLA4','CXCR6','ITGAE','CD38','PDCD1','CXCL13','TOX','DUSP4','TNFRSF9','TIGIT','LAYN','RTKN2', 'FOXP3', 'FANK1','IL2RA']

In [ ]:
goi=['CDCA2', 'MKI67', 'STMN1','LRRN3', 'CCR7','NPM1', 'SELL', 'IL7R', 'TCF7', 'KLRB1', 'FOXP3', 'RTKN2', 'IL2RA',
'CXCR5', 'CXCL13', 'SLAMF6', 'TOX', 'RUNX3', 'PRDM1', 'IFNGR1','KLRG1','ZNF683','FCRL6','CXCR6',
'IFIT1', 'IFITM1', 'ISG15', 'XCL1','XCL2', 'CD69', 'FOS','GZMK','CCL5', 'CCL4','GZMH', 'GZMB', 'GZMA','GNLY',
     'NKG7', 'PRF1', 'PRDM1', 'IFNG','FANK1', 'RSAD2', 'MX1', 'GZMA', 'IL4I1', 'JUN',  'IGFL2', 
     'IL21', 'CAV1', 'PDCD1', 'LAG3', 'HAVCR1', 'ENTPD1', 'CTLA4', 'ITGAE','CD38','TNFRSF9','TIGIT','HSPA1A', 'HSPA1B', 'DUSP4', 'ATF3','DNAJB1']

#LRRN3, CCR7, NPM1, SELL, PASK, IL7R, KLRB1, TNFSF13B, GZMK, CCL5, CCL4, GZMH, GZMA, GNLY, NGK7, CST7, PRDM1, IFNG, DNAJB1, HSPA1A, HSPA1B, 

In [ ]:
goi=[ 'CCR7', 'SELL', 'IL7R', 'TCF7', 'KLRB1',
     'FOXP3', 'RTKN2', 'IL2RA', 'PRDM1', 'RUNX3',  'IFNGR1',
'IFIT1', 'IFITM1', 'ISG15','CDCA2', 'MKI67', 'STMN1','CXCR5', 'CXCL13', 'SLAMF6', 'TOX', 'PDCD1','HAVCR1','LAG3',  'ENTPD1', 'CTLA4', 'ITGAE','PRF1', 
     'XCL1', 'CD69', 'FOS', 'GZMB', 'GZMA',  'HSPA1A', 'HSPA1B', 'ATF3']

In [ ]:
root_path = os.getcwd()
bescapath_full = os.path.dirname(bc.__file__)
bescapath = os.path.split(bescapath_full)[0]

In [ ]:
gmt_file_anno= bescapath + '/besca/datasets/genesets/CellNames_scseqCMs6_sigs.gmt'

mymarkers = bc.tl.sig.read_GMT_sign(gmt_file_anno,directed=False)
mymarkers = bc.tl.sig.filter_siggenes(adata, mymarkers) 

In [ ]:
cd4sigmarkers=mymarkers['CD4Tcell']+mymarkers['CD4Tcell_IL7Rmax']+mymarkers['NaiCD4Tcell']+mymarkers['CMCD4Tcell']+mymarkers['EMCD4Tcell']+mymarkers['RegTcell']+mymarkers['CytotoxCD4Tcell']+mymarkers['ProlifCD4Tcell']

In [ ]:
#['CCR7', 'IL7R', 'S1PR1', 'KLF2', 'NKG7', 'IFNGR1', 'RUNX3', 'PRDM1', 'SLAMF6', 'TOX', 'BCL2', 'PSGL1', 'LY6C', 'CXCR6', 'CD27', 'HAVCR2', 'FR4', 'CXCR5', 'CCR7', 'IL7R', 'CD69', 'GZMB', 'GZMK']

In [ ]:
cd4m=list(set(cd4sigmarkers ).union(set(['CCR7', 'IL7R', 'S1PR1', 'KLF2', 'NKG7', 'IFNGR1', 'RUNX3', 'PRDM1', 'SLAMF6', 'TOX', 'BCL2', 'CXCR6', 'CD27', 'HAVCR2', 'CXCR5', 'CCR7', 'IL7R', 'CD69', 'GZMB', 'GZMK'])))
   

In [ ]:
#average_obs,fraction_obs=bc.get_means(tmp, 'leiden')

In [ ]:
sc.pl.umap(subdata, color='leiden')

In [ ]:
average_obs,fraction_obs=bc.get_means(tmp, 'leiden')

In [ ]:
### Plot parameters for publication 
def set_pub():    
    small_size = 10
    medium_size = 12
    large_size = 14

    resolution = 300 #in dpi
    plt.rcParams['font.weight'] = 'normal'
    #plt.rc('font', **{'family':'sans-serif','sans-serif':['Helvetica']})
    plt.rc('axes', titlesize=large_size, titleweight = "bold")               # fontsize of the axes title
    plt.rc('axes', labelsize=medium_size, labelweight = "bold")               # fontsize of the x and y labels
    plt.rc('xtick', labelsize=small_size)               # fontsize of the tick labels
    plt.rc('ytick', labelsize=small_size)               # fontsize of the tick labels
    plt.rc('legend', fontsize=small_size, title_fontsize = medium_size)               # legend fontsize
    plt.rc('figure', titlesize=large_size, titleweight = "bold")              # fontsize of the figure title
    plt.rc('savefig', dpi=resolution)                   # higher res outputs

    plt.rcParams['svg.fonttype'] = 'none'


set_pub()

In [ ]:
a=sns.clustermap(average_obs.loc[myorder,cd4m],col_cluster=True, row_cluster=False,figsize=(20,5),
                 cmap='viridis',metric='cosine', standard_scale=1)


In [ ]:
a=sns.clustermap(average_obs.loc[myorders,goi],col_cluster=True, row_cluster=False,figsize=(20,5),
                 cmap='viridis',metric='cosine', standard_scale=1)
a.savefig(figdir+'Heatmap-goi-'+velosubset+'.leiden.all.standard.pdf')
a.savefig(figdir+'Heatmap-goi-'+velosubset+'.leiden.all.standard.svg', format='svg', bbox_inches="tight", dpi=300)

In [ ]:
sc.set_figure_params(14)

In [ ]:
a=sns.clustermap(average_obs.loc[myorder,goi],col_cluster=False, row_cluster=False,figsize=(20,5),
                 cmap='viridis',metric='cosine', standard_scale=1)
a.savefig(figdir+'Heatmap-goi-'+velosubset+'.leiden.all.standard.notclustered.pdf')
a.savefig(figdir+'Heatmap-goi-'+velosubset+'.leiden.all.standard.notclustered.svg', format='svg', bbox_inches="tight", dpi=300)

In [ ]:
a=sns.clustermap(average_obs.loc[myorders,goi],col_cluster=False, row_cluster=False,figsize=(20,5),
                 cmap='viridis',metric='cosine', standard_scale=1)
a.savefig(figdir+'Heatmap-goi-'+velosubset+'.leiden.subdata.standard.notclustered.pdf')
a.savefig(figdir+'Heatmap-goi-'+velosubset+'.leiden.subdata.standard.notclustered.svg', format='svg', bbox_inches="tight", dpi=300)

In [ ]:
tmp.obs['celltype_new']=tmp.obs['leiden'].copy()

In [ ]:
tmp.obs['celltype_new']=tmp.obs['celltype_new'].replace('4', 'naive')
tmp.obs['celltype_new']=tmp.obs['celltype_new'].replace('3', 'naive')
tmp.obs['celltype_new']=tmp.obs['celltype_new'].replace('1', 'central memory')
tmp.obs['celltype_new']=tmp.obs['celltype_new'].replace('0', 'central memory')
tmp.obs['celltype_new']=tmp.obs['celltype_new'].replace('11', 'central memory')
tmp.obs['celltype_new']=tmp.obs['celltype_new'].replace('2', 'effector memory')
tmp.obs['celltype_new']=tmp.obs['celltype_new'].replace('5', 'effector memory, cellular stress')
tmp.obs['celltype_new']=tmp.obs['celltype_new'].replace('13', 'activated, IFN')
tmp.obs['celltype_new']=tmp.obs['celltype_new'].replace('6', 'effector')
tmp.obs['celltype_new']=tmp.obs['celltype_new'].replace('8', 'follicular helper')
tmp.obs['celltype_new']=tmp.obs['celltype_new'].replace('9', 'proliferating')
tmp.obs['celltype_new']=tmp.obs['celltype_new'].replace('10', 'central memory/Treg')
tmp.obs['celltype_new']=tmp.obs['celltype_new'].replace('7', 'Treg')
tmp.obs['celltype_new']=tmp.obs['celltype_new'].replace('12', 'Treg')

In [ ]:
sc.pl.umap(tmp, color='celltype_new', save='-velo-'+velosubset+'-celltype_new.svg')

In [ ]:
a=sns.clustermap(average_obs.loc[:,goi],col_cluster=True, row_cluster=True,figsize=(20,5),
                 cmap='viridis',metric='cosine', standard_scale=1)


In [ ]:
### Generate plots of the top marker genes 
sc.pl.dotplot(tmp, var_names=goi, groupby=labeloi, dot_max=0.8,vmax=3, dendrogram=True)


In [ ]:
a=sns.clustermap(average_obs.loc[myorder,cd4m],col_cluster=True, row_cluster=False,figsize=(18,5),
                 cmap='viridis',metric='cosine', standard_scale=1)
#a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.tdata.standard.pdf')
#a.savefig(figdir+'Heatmap-goi-'+velosubset+'.celltype3_pub.tdata.standard.svg', format='svg', bbox_inches="tight", dpi=300)

In [ ]:
#['ZBTB16','IL7R','BCL2','MAL','LEF1','TCF7','S1PR1','KLF2','NELL2','AQP3','GNLY','HMGB2','STMN1','MKI67',
#'SAMD3','MATK','IL18RAP','GZMK','GZMA','KLRD1','']

In [ ]:
#myorder=['3', '4', '5', '0', '13', '1', '10', '9', '2', '8', '7', '6', '11','12']

In [ ]:
sc.pl.matrixplot

In [ ]:
#categories_order=myorder,
sc.pl.matrixplot(tmp,var_names=goi,groupby='leiden',dendrogram=False, 
                 standard_scale='var', categories_order=myorder,
                 save='Heatmap-goi-'+velosubset+'.leiden.tdata.nc.scanpy_zscore.svg')

In [ ]:
#categories_order=myorder,
sc.pl.matrixplot(tmp,var_names=cd4m,groupby='leiden',dendrogram=False, 
                 standard_scale='var', categories_order=myorder,
                 save='Heatmap-goi-'+velosubset+'.leiden.tdata.nc.scanpy_zscore.svg')

In [ ]:
#categories_order=myorder,
#sc.pl.matrixplot(tmp,var_names=goi,groupby='leiden',dendrogram=False, 
#                 standard_scale='var', categories_order=myorder,
#                 save='Heatmap-goi-'+velosubset+'.leiden.tdata.nc.scanpy_zscore.svg')

In [ ]:
#categories_order=myorder,
sc.pl.matrixplot(tmp,var_names=goi,groupby='celltype3_pub',dendrogram=False, 
                 standard_scale='var', 
                 save='Heatmap-goi-'+velosubset+'.celltype3_pub.tdata.nc.scanpy_zscore.svg')

In [ ]:
###subdata.uns['iroot'] = np.flatnonzero(((subdata.obs['leiden']  == '3')&(subdata.obs['root_cells']>0.8)&(subdata.obs['Sample type']  == 'PBMC')))[0]
#adata.uns['iroot'] =np.flatnonzero(((adata.obs['root_cells']>0.8)&((adata.obs['celltype3']  == 'naive thymus-derived CD8-positive, alpha-beta T cell')|(adata.obs['celltype3_TNK']  == 'NaiCD8Tcell'))))[0]
###root_key='iroot'

In [ ]:
###scv.tl.recover_latent_time(subdata, root_key='iroot', end_key='terminal_states_probs')

In [ ]:
###scv.pl.scatter(subdata, color='latent_time', color_map='gnuplot', size=80)

In [ ]:
#scv.tl.recover_latent_time(subdata, root_key='initial_states_probs', end_key='terminal_states_probs')

In [ ]:
scv.tl.paga(subdata, groups='leiden', root_key='iroot', end_key='iend') #use_time_prior='velocity_pseudotime'

In [ ]:
#import cellrank as cr

In [ ]:
sc.set_figure_params(10)
scv.pl.paga(subdata,cluster_key="leiden", basis='umap',node_size_scale=1, edge_width_scale=1)

In [ ]:
#scv.pl.velocity_graph(subdata, threshold=.1)

In [ ]:
#scv.tl.velocity_pseudotime(subdata)
#scv.pl.scatter(subdata, color='velocity_pseudotime', cmap='gnuplot')

In [ ]:
subdata.obs['clusters']=subdata.obs.leiden.copy()

In [ ]:
top_genes = subdata.var['fit_likelihood'].sort_values(ascending=False).index[:50]
scv.pl.heatmap(subdata, var_names=top_genes, sortby='latent_time', col_color='clusters',
               n_convolve=100,font_scale=0.6)

In [ ]:
top_genes = subdata.var['fit_likelihood'].sort_values(ascending=False).index[:50]
scv.pl.heatmap(subdata, var_names=top_genes, sortby='latent_time', col_color='celltype3_pub',
               n_convolve=100,font_scale=0.5)

In [ ]:
scv.tl.rank_dynamical_genes(subdata, groupby='clusters')
df = scv.get_df(subdata, 'rank_dynamical_genes/names')
df.head(5)

In [ ]:
scv.pp.filter_and_normalize(rbdata,min_shared_counts=20, n_top_genes=800)
scv.pp.neighbors(rbdata)
scv.pp.moments(rbdata)

scv.tl.recover_dynamics(rbdata, n_jobs=20)
scv.tl.velocity(rbdata,mode='dynamical')
scv.tl.velocity_graph(rbdata)


In [ ]:
scv.tl.velocity_graph(rbdata) #mode_neighbors='connectivities' 

In [ ]:
scv.tl.velocity_confidence(rbdata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(rbdata, c=keys, cmap='coolwarm', perc=[5, 95])

scv.pl.velocity_embedding_stream(rbdata, basis='umap',color=['Sample type'], 
                                 save='-velo-'+velosubset+'-dyn-Sample_type.rbdata.svg')

scv.pl.velocity_embedding_stream(rbdata, basis='umap',color=['leiden'], 
                                 save='-velo-'+velosubset+'-dyn-leiden.rbdata.svg')

scv.tl.recover_latent_time(rbdata)

scv.pl.scatter(rbdata, color='latent_time', color_map='gnuplot', size=80)

In [ ]:
scv.pl.scatter(rbdata, color='velocity_pseudotime', color_map='gnuplot', size=80)

In [ ]:
rbdata.obs.loc[((rbdata.obs['root_cells']>0.5) & (rbdata.obs['Sample type']  == 'PBMC')),'leiden'].value_counts()

In [ ]:
rbdata.obs.loc[((rbdata.obs['end_points']>0.9) & (rbdata.obs['Sample type']  == 'TIL')),'leiden'].value_counts()

In [ ]:
rbdata.uns['iroot'] = np.flatnonzero(((rbdata.obs['leiden']  == '13')&(rbdata.obs['root_cells']>0.5)&(rbdata.obs['Sample type']  == 'PBMC')))[0]
rbdata.uns['iend'] = np.flatnonzero(((rbdata.obs['leiden']  == '1')&(rbdata.obs['end_points']>0.9)&(rbdata.obs['Sample type']  == 'TIL')))[0]


In [ ]:

scv.tl.recover_latent_time(rbdata, root_key='iroot', end_key='iend')

scv.pl.scatter(rbdata, color='latent_time', color_map='gnuplot', size=80)

In [ ]:
sc.pl.umap(rbdata, color='velocity_pseudotime')

In [ ]:
scv.tl.paga(rbdata, groups='leiden', root_key='iroot', end_key='iend', 
            use_time_prior='velocity_pseudotime', threshold_root_end_prior=0.8)


In [ ]:
sc.set_figure_params(10)
scv.pl.paga(rbdata,cluster_key="leiden", basis='umap',node_size_scale=1, edge_width_scale=1)

In [ ]:
sc.pl.umap(rbdata, color=['root_cells','end_points'])

In [ ]:
scv.pp.filter_and_normalize(nrbdata,min_shared_counts=20, n_top_genes=1000)
scv.pp.neighbors(nrbdata)
scv.pp.moments(nrbdata)

scv.tl.recover_dynamics(nrbdata, n_jobs=20)
scv.tl.velocity(nrbdata,mode='dynamical')
#scv.tl.velocity_graph(nrbdata)
scv.tl.velocity_graph(nrbdata, mode_neighbors='connectivities') 


scv.tl.velocity_confidence(nrbdata)
keys = 'velocity_length', 'velocity_confidence'
scv.pl.scatter(nrbdata, c=keys, cmap='coolwarm', perc=[5, 95])

scv.pl.velocity_embedding_stream(nrbdata, basis='umap',color=['Sample type'], 
                                 save='-velo-'+velosubset+'-dyn-Sample_type.nrbdata.svg')

scv.pl.velocity_embedding_stream(nrbdata, basis='umap',color=['leiden'], 
                                 save='-velo-'+velosubset+'-dyn-leiden.nrbdata.svg')

scv.tl.recover_latent_time(nrbdata)

scv.pl.scatter(nrbdata, color='latent_time', color_map='gnuplot', size=80)


In [ ]:
nrbdata.obs.loc[((nrbdata.obs['root_cells']>0.9) & (nrbdata.obs['Sample type']  == 'PBMC')),'leiden'].value_counts()


In [ ]:

nrbdata.obs.loc[((nrbdata.obs['end_points']>0.9) & (nrbdata.obs['Sample type']  == 'TIL')),'leiden'].value_counts()

In [ ]:
nrbdata.uns['iroot'] = np.flatnonzero(((nrbdata.obs['leiden']  == '0')&(nrbdata.obs['root_cells']>0.9)&(nrbdata.obs['Sample type']  == 'PBMC')))[0]
nrbdata.uns['iend'] = np.flatnonzero(((nrbdata.obs['leiden']  == '5')&(nrbdata.obs['end_points']>0.9)&(nrbdata.obs['Sample type']  == 'TIL')))[0]

scv.tl.recover_latent_time(nrbdata, root_key='iroot', end_key='iend')

scv.pl.scatter(nrbdata, color='latent_time', color_map='gnuplot', size=80)

In [ ]:

scv.tl.paga(nrbdata, groups='leiden', root_key='iroot', end_key='iend', 
            use_time_prior='velocity_pseudotime')


sc.set_figure_params(10)
scv.pl.paga(nrbdata,cluster_key="leiden", basis='umap',node_size_scale=1, edge_width_scale=1)

In [ ]:
sc.pl.umap(nrbdata, color=['root_cells','end_points'])

In [ ]:
sc.pl.umap(nrbdata, color=['latent_time','velocity_pseudotime'])

In [ ]:
#run cellrankanalysis
cellrankanalysis=False
if cellrankanalysis:
    g = cr.estimators.GPCCA(vk)
    g.fit(cluster_key="clusters", n_states=[2, 6])
    g.plot_macrostates(which="all", discrete=True, legend_loc="right", s=100)

    g.predict_terminal_states()
    g.plot_macrostates(which="terminal", legend_loc="right", s=100)

    g.plot_macrostates(which="terminal", discrete=False)

    g.predict_initial_states()
    g.plot_macrostates(which="initial", legend_loc="right", s=100)


    g.fit(cluster_key="clusters", n_states=11)

    g.set_terminal_states(states=["Alpha", "Beta", "Epsilon", "Delta"])
    g.plot_macrostates(which="terminal", legend_loc="right", size=100)


    g.compute_fate_probabilities()
    g.plot_fate_probabilities(same_plot=False)
    g.plot_fate_probabilities(same_plot=True)

    driver_clusters = ["Delta", "Fev+", "Ngn3 high EP", "Ngn3 low EP"]

    delta_df = g.compute_lineage_drivers(
        lineages=["Delta"], cluster_key="clusters", clusters=driver_clusters
    )
    delta_df.head(10)

    sc.pl.violin(adata, keys=["palantir_pseudotime"], groupby="clusters", rotation=90)

    adata.obs["fate_probabilities_delta"] = g.fate_probabilities["Delta"].X.flatten()

    sc.pl.embedding(
        adata,
        basis="umap",
        color=["fate_probabilities_delta"] + list(delta_df.index[:8]),
        color_map="viridis",
        s=50,
        ncols=3,
        vmax="p96",
    )


In [ ]:
sc.pl.umap(subdata, color=['root_cells','end_points'])

In [ ]:
subdata.obs['clusters']=subdata.obs['leiden'].copy()

In [ ]:
top_genes = subdata.var['fit_likelihood'].sort_values(ascending=False).index[:20]

In [ ]:
sc.set_figure_params()

In [ ]:
degenes=['PRF1', 'TMEM258', 'SPP1', 'MIF', 'HLA-DRB5', 'ATP5I', 'GZMB', 'PPDPF', 'PDCD1', 'MT2A', 
          'TUBA1A', 'PTGER4', 'JUNB', 'SERTAD1', 'TNFAIP3', 'PNRC1']

In [ ]:
sc.pl.umap(subdata, color=degenes)

In [ ]:
sc.pl.umap(subdata, color=top_genes[0:12])

In [ ]:
sc.pl.umap(subdata, color=top_genes[11:20])

In [ ]:
tmp=subdata[subdata.obs.leiden.isin(['2','10','8','7'])]

In [ ]:
sc.pl.umap(tmp, color='PatientID')

In [ ]:
sc.pl.umap(tmp, color='RCat')

In [ ]:
tmp.obs.PatientID='P'+tmp.obs.PatientID.astype(str)

In [ ]:
clustercomp=bc.tl.count_occurrence_subset(tmp, 'PatientID', count_variable ='leiden', return_percentage = True)

In [ ]:
sc.set_figure_params()
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (12,5)
import matplotlib 

#clustercomp=clustercomp[clustercomp.max().sort_values().index].copy()
#clustercomp.transpose().plot(kind='bar', stacked=True)
clustercomp[clustercomp.max().sort_values().index].transpose().plot.bar(stacked=True, figsize=(12, 8))
plt.ylabel('Percentage donor per cluster')
plt.legend(loc='center left', bbox_to_anchor=(1, 0.5))
#fig.savefig(figdir+'Clustercomp-'+velosubset+'.PatientID.pdf'),bbox_inches='tight')
plt.savefig(figdir+'Clustercomp-'+velosubset+'.PatientID.svg', format='svg', bbox_inches="tight", dpi=300)

In [ ]:
top_genes = subdata.var['fit_likelihood'].sort_values(ascending=False).index[:30]
scv.pl.heatmap(subdata, var_names=top_genes, sortby='latent_time', col_color='clusters',
               font_scale=0.5)

In [ ]:
#scv.pl.velocity_embedding_stream(subdata, vkey=[ 'velocity_random'],color=['celltype3_pub'],legend_loc='right')

In [ ]:
scv.tl.score_genes_cell_cycle(adata)
scv.pl.scatter(adata, color_gradients=['S_score', 'G2M_score'], smooth=True, perc=[5, 95])
s_genes, g2m_genes = scv.utils.get_phase_marker_genes(adata)


In [ ]:
import cellrank as cr

In [ ]:
import warnings

warnings.simplefilter("ignore", category=UserWarning)

In [ ]:
vk = cr.kernels.VelocityKernel(subdata)


In [ ]:
vk.compute_transition_matrix()

In [ ]:
ck = cr.kernels.ConnectivityKernel(subdata)
ck.compute_transition_matrix()

In [ ]:
ck.plot_projection()

In [ ]:
vk.plot_projection()

In [ ]:
vk.plot_random_walks(start_ixs={"leiden": "3"}, max_iter=200, seed=0,    dpi=150)

In [ ]:
ck.plot_random_walks(start_ixs={"leiden": "3"}, max_iter=200, seed=0,    dpi=150)

In [ ]:
vk.plot_projection(basis="umap")

In [ ]:
top_genes

In [ ]:
top_genes = subdata.var['fit_likelihood'].sort_values(ascending=False).index[:30]
scv.pl.heatmap(subdata, var_names=top_genes, sortby='latent_time', col_color='clusters',
               font_scale=0.5)

In [ ]:
top_genes = subdata.var["fit_likelihood"].sort_values(ascending=False).index
scv.pl.scatter(subdata, basis=top_genes[:10], ncols=5, frameon=False)

In [ ]:
vk.write_to_adata()
subdata.write(
    "Velo-"+velosubset+"_velocity_kernel.h5ad", compression="gzip"
)

In [ ]:
mysubset='All'
subcat='PBMCandTIL'


In [ ]:
#subcat='TILonly'
subdata.write(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')

In [ ]:
subdata=sc.read(WD+'analyzed/'+analysis_name+'/velocity/Velo-'+velosubset+'-'+mysubset+'-'+subcat+'-dyn.h5ad')

In [ ]:
ddata=subdata.copy()

In [ ]:
ddata.obs['RCat']=ddata.obs['Response3'].copy()
ddata.obs['RCat']=list(ddata.obs['RCat'].replace('NR', 'NR_nadj'))
ddata.obs['RCat']=list(ddata.obs['RCat'].replace('PD', 'NR_adj'))

In [ ]:
sc.pl.umap(ddata, color='leiden',legend_loc='on data')

In [ ]:
sc.pl.umap(ddata, color='celltype3_pub')

In [ ]:
set(ddata.obs['PatientID'])

In [ ]:
### Remove two uncertain patients

#ddata=ddata[ddata.obs['PatientID']!='P38'].copy()
#ddata=ddata[ddata.obs['PatientID']!='P12'].copy()

In [ ]:
ddata=ddata[ddata.obs['Sample type']=='TIL'].copy()

In [ ]:
#ddata=ddata[ddata.obs['celltype3_pub']!='naive CD4-positive T cell'].copy()

In [ ]:
#set(ddata.obs.celltype3_pub)

In [ ]:
ddata.obs['PatientID']=ddata.obs['PatientID'].astype(str)

In [ ]:
ddata.obs['R_clus']=ddata.obs['RCat'].astype(str)+'-'+[x.replace("-", "_") for x in ddata.obs['leiden'].astype(str)]
ddata.obs['Pat_clus']=ddata.obs['PatientID'].astype(str)+'-'+[x.replace("-", "_") for x in ddata.obs['leiden'].astype(str)]
dfPat = ddata.obs.groupby('Pat_clus')['velocity_length', 'velocity_confidence','latent_time'].mean().T

In [ ]:
dfPat

In [ ]:
Resp=[ddata[ddata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0] for x in list(dfPat.columns)]
PatientID=[x.split('-')[0]  for x in list(dfPat.columns)]
ClusterID=[x.split('-')[1]  for x in list(dfPat.columns)]

In [ ]:
mydf=pd.DataFrame([list(dfPat.loc['velocity_length',:]),Resp,PatientID,ClusterID]).transpose()
mydf.index=dfPat.columns
mydf.columns=['velocity_length','Response','PatientID','Cluster']

mydfl=pd.DataFrame([list(dfPat.loc['latent_time',:]),Resp,PatientID,ClusterID]).transpose()
mydfl.index=dfPat.columns
mydfl.columns=['latent_time','Response','PatientID','Cluster']

In [ ]:
mydf.groupby('PatientID', as_index=False)['velocity_length']

In [ ]:
color_dict = {'R': 'coral', 'TF': 'firebrick', 'NR_nadj': 'lightskyblue','NR_adj': 'royalblue'}

#### Velocity test first

In [ ]:
#corder=['8','6','1','21']
mydf['velocity_length']=mydf['velocity_length'].astype('float')
velMean=mydf.groupby('PatientID')['velocity_length'].mean()
patid=[ddata[ddata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0]  for x in list(velMean.index)]
totest=pd.DataFrame([list(velMean),patid]).transpose()
totest.columns=['Val','Cat']

In [ ]:
ss.mannwhitneyu(list(totest.loc[totest['Cat']=='R',:]['Val']),list(totest.loc[totest['Cat']=='NR_nadj',:]['Val']))

In [ ]:
ss.mannwhitneyu(list(totest.loc[totest['Cat']=='TF',:]['Val']),list(totest.loc[totest['Cat']=='NR_adj',:]['Val']))

In [ ]:
set(mydf['Response'])

In [ ]:
totestk=totest.copy()

In [ ]:
mysubset='Final'

In [ ]:
subcat='CD4T'

In [ ]:
fig=sns.boxplot(y='Val',x='Cat',data=totestk,palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
fig=sns.swarmplot(x='Cat',y='Val',data=totestk,color='black',order=['R','TF','NR_nadj','NR_adj'])
fig.figure.savefig(figdir+'Velocitylen-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.pdf') 

In [ ]:
totest=list(itertools.combinations(['R','TF','NR_nadj','NR_adj'], 2))
pwilc={}
pt={}
for pairs in totest:
    pwilc[pairs[0]+'-'+pairs[1]]=stats.mannwhitneyu(list(totestk.loc[totestk['Cat']==pairs[0],:]['Val']), 
                           list(totestk.loc[totestk['Cat']==pairs[1],:]['Val']))[1]
    pt[pairs[0]+'-'+pairs[1]]=stats.ttest_ind(list(totestk.loc[totestk['Cat']==pairs[0],:]['Val']), 
                           list(totestk.loc[totestk['Cat']==pairs[1],:]['Val']))[1]

myps=pd.DataFrame([pwilc,pt]).transpose()
myps.columns=['MannWhitney','T-test']

myps.to_csv(figdir+'Pval-Velocitylen-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.pdf',sep='\t')

In [ ]:
myps

In [ ]:
corder=list(set(mydf['Cluster']))

In [ ]:
for i in corder:
    sns.boxplot(y='velocity_length',x='Response',data=mydf.loc[mydf['Cluster']==i,:],
                palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
    sns.swarmplot(x='Response',y='velocity_length',data=mydf.loc[mydf['Cluster']==i,:],
                  color='black',order=['R','TF','NR_nadj','NR_adj'])
    print('Cluster '+i)
    plt.figure()

#### Latent time next

In [ ]:
mydfl

In [ ]:
#corder=['8','6','1','21']
mydfl['latent_time']=mydfl['latent_time'].astype('float')
velMean=mydfl.groupby('PatientID')['latent_time'].mean()
patid=[ddata[ddata.obs['PatientID']==x.split('-')[0]].obs['RCat'][0]  for x in list(velMean.index)]
totest=pd.DataFrame([list(velMean),patid]).transpose()
totest.columns=['Val','Cat']

ss.mannwhitneyu(list(totest.loc[totest['Cat']=='R',:]['Val']),list(totest.loc[totest['Cat']=='NR_nadj',:]['Val']))



In [ ]:
ss.mannwhitneyu(list(totest.loc[totest['Cat']=='TF',:]['Val']),list(totest.loc[totest['Cat']=='NR_adj',:]['Val']))

In [ ]:
totestk=totest.copy()

In [ ]:
fig=sns.boxplot(y='Val',x='Cat',data=totestk,palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
fig=sns.swarmplot(x='Cat',y='Val',data=totestk,color='black',order=['R','TF','NR_nadj','NR_adj'])
fig.figure.savefig(figdir+'LatentTime-'+velosubset+'-'+mysubset+'-'+subcat+'-per-response.pdf') 

In [ ]:
corder=list(set(mydf['Cluster']))

for i in corder:
    sns.boxplot(y='latent_time',x='Response',data=mydfl.loc[mydf['Cluster']==i,:],
                palette=color_dict,order=['R','TF','NR_nadj','NR_adj'])
    sns.swarmplot(x='Response',y='latent_time',data=mydfl.loc[mydfl['Cluster']==i,:],
                  color='black',order=['R','TF','NR_nadj','NR_adj'])
    plt.title('Cluster ' +i)
    print('Cluster '+i)
    plt.figure()

In [ ]:
velogenes=subdata.var.loc[(subdata.var['velocity_genes']==True)&(subdata.var['fit_likelihood']>.1),:]

In [ ]:
velogenes.sort_values('fit_likelihood', ascending=False)

In [ ]:
set(list(velogenes.index))

In [ ]:
top_genes = subdata.var['fit_likelihood'].sort_values(ascending=False).index#[:300]


In [ ]:
invelo=[]
for x in list(top_genes):
    invelo=invelo+list(set([x]).intersection(set(velogenes.index)))

In [ ]:
scv.pl.heatmap(subdata, var_names=invelo, sortby='latent_time', 
               col_color='leiden', n_convolve=30,
               font_scale=0.5,  figsize=(8, 6))

In [ ]:
scv.pl.heatmap(subdata, var_names=invelo[0:50], 
               sortby='latent_time', col_color='leiden', n_convolve=30,
               font_scale=0.5,  figsize=(8, 6))

In [ ]:
scv.pl.heatmap(subdata, var_names=invelo[50:100], 
               sortby='latent_time', col_color='leiden', n_convolve=30,
               font_scale=0.5,  figsize=(8, 6))

In [ ]:
if (len(invelo)>100):
    scv.pl.heatmap(subdata, var_names=invelo[100:len(invelo)], 
               sortby='latent_time', col_color='leiden', n_convolve=30,
               font_scale=0.5,  figsize=(8, 6))

In [ ]:
velogenes.sort_values('fit_likelihood', ascending=False).to_csv(figdir+'/VelocityInfoPerGene_'+velosubset+'-'+mysubset+'_'+subcat+'_loom-dyn_allVeloGenesRanked.tsv',sep='\t')

In [ ]:
tkey='latent_time'
var_names=invelo
var_names = [name for name in var_names if name in adata.var_names]
xkey='Ms'
time = subdata.obs[tkey].values
time = time[np.isfinite(time)]

df = pd.DataFrame(subdata[:, var_names].layers[xkey][np.argsort(time)], columns=var_names)

In [ ]:
max_sort = np.argsort(np.argmax(df.values, axis=0))
df = pd.DataFrame(df.values[:, max_sort], columns=df.columns[max_sort])
df.index=pd.DataFrame([subdata.obs['celltype3_merged'][np.argsort(time)],
                       subdata.obs['leiden'][np.argsort(time)],subdata.obs['latent_time'][np.argsort(time)]]).transpose().index
df=pd.concat([df,pd.DataFrame([subdata.obs['celltype3_merged'][np.argsort(time)],
                               subdata.obs['leiden'][np.argsort(time)],subdata.obs['latent_time'][np.argsort(time)]]).transpose()], axis=1)

In [ ]:
def getMeans(adata,myvar):
    gene_ids = adata.raw.var.index.values
    clusters = adata.obs[myvar].cat.categories
    obs = adata.raw[:,gene_ids].X.toarray()
    obs = pd.DataFrame(obs,columns=gene_ids,index=adata.obs[myvar])
    average_obs = obs.groupby(level=0).mean()
    obs_bool = obs.astype(bool)
    fraction_obs = obs_bool.groupby(level=0).sum()/obs_bool.groupby(level=0).count()
    return(average_obs,fraction_obs)


In [ ]:
avg,fct=getMeans(adata,'leiden')

In [ ]:
! jupyter nbconvert --to html scvelo-CD4T.ipynb